# Downsampling and BIDS conversion

## Introduction

In this section we will demonstrate how to prepare the data for further analysis. As the FieldLine data are sampled at 5000 Hz the first step is to downsample to 1000 Hz. Subsequenctly we will convert the data to MEG BIDS (Niso et al., 2018). BIDS (Brain Imaging Data Structure) is a standardised format for organising and describing neuroimaging datasets. It ensures that the meta data are present and that the file structure is consistent over data sets. 

## Preparation

Import the required modules:

In [1]:
import os

import numpy as np
import pandas as pd

import mne
from mne_bids import (
    BIDSPath,
    make_dataset_description,
    print_dir_tree,
    read_raw_bids,
    write_meg_calibration,
    write_meg_crosstalk,
    write_raw_bids,
)

### File overview

The chapter relies on the input files:
~~~
<ROOT>/Main.fif
~~~ 
and generate the out files:
~~~
<ROOT>/Main_fs.fif
<ROOT>/Main_event.fif

<ROOT/Sub3_BIDS/participants.json
<ROOT/Sub3_BIDS/participants.tsv
<ROOT/Sub3_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv
<ROOT/Sub3_BIDS/dataset_description.json
<ROOT/Sub3_BIDS/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json
<ROOT/Sub3_BIDS/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_channels.tsv
<ROOT/Sub3_BIDS/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_events.json
<ROOT/Sub3_BIDS/meg/sub-01_ses-01_task-SpAtt_run-1_events.tsv
<ROOT/Sub3_BIDS/meg/sub-01_ses-01_task-SpAtt_run-1_meg.fif
<ROOT/Sub3_BIDS/sub-01_ses-01_task-SpAtt_run-1_meg.json
~~~

### Loading the data

The FieldLine data are stored in FIF format which is a binary file structure with embedded lables.

As a first step we will load the data. Make sure the data_data path is set to where the data are stored. We will also define the name of the resampled data with 'rs' added to the filename ( <a>  *_rs.fif  </a>).


In [2]:
data_path = '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3'
file_name = 'Main.fif'
raw_fname = os.path.join(data_path, file_name)

raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw_resampled_fname = raw_fname.replace('.fif', f'_rs.fif')

event_fname = raw_fname.replace('.fif', f'_event.fif')

bids_folder = os.path.join(data_path, "Sub3_BIDS")

Opening raw data file /home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main.fif...


/tmp/ipykernel_39257/2542081543.py:5: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_fname, preload=True)


    Range : 0 ... 8861899 =      0.000 ...  1772.380 secs
Ready.
Reading 0 ... 8861899  =      0.000 ...  1772.380 secs...


# Lowpass filtering and resampling

The data will now be resampled to 1000 Hz. Prior to resampling data will be lowpass filtered at 1000 Hz/4 = 250 Hz. The downsample is done to avoid alias problems (see Smith 1998, chapter 3). After the lowpass filtering the data are resampled to the desired frequency.

In [3]:
desired_sfreq = 1000
current_sfreq = raw.info['sfreq']

lowpass_freq = desired_sfreq / 4.0
raw_resampled = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)

raw_resampled.resample(sfreq=desired_sfreq)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 2.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 250.00 Hz
- Upper transition bandwidth: 62.50 Hz (-6 dB cutoff frequency: 281.25 Hz)
- Filter length: 265 samples (0.053 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.5s


1744 events found on stim channel di32
Event IDs: [   1    2    6    7   50  400  500  800  900 1000 1100 2000 2100]
1744 events found on stim channel di32
Event IDs: [   1    2    6    7   50  400  500  800  900 1000 1100 2000 2100]


<Raw | Main.fif, 71 x 1772380 (1772.4 s), ~960.1 MB, data loaded>

##### As a final step, the resampled data will be saved

In [4]:
raw_resampled.save(raw_resampled_fname, overwrite=True)

Writing /home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_rs.fif


/tmp/ipykernel_39257/649465886.py:1: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_resampled.save(raw_resampled_fname, overwrite=True)


Closing /home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_rs.fif
[done]


To verify that the data has been resampled write

In [5]:
print(raw_resampled.info)

<Info | 17 non-empty values
 bads: []
 ch_names: L102_bz-s73, L104_bz-s80, L106_bz-s84, L108_bz-s77, L110_bz-s76, ...
 chs: 70 Magnetometers, 1 Stimulus
 custom_ref_applied: False
 description: {"chassis":{"version":"0.9.4- ...
 dig: 6 items (3 Cardinal, 3 HPI)
 experimenter: AK
 file_id: 4 items (dict)
 gantry_angle: 0.0
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 250.0 Hz
 meas_date: 2024-12-06 18:13:30 UTC
 meas_id: 4 items (dict)
 nchan: 71
 proj_id: 1 item (ndarray)
 proj_name: Empty
 projs: []
 sfreq: 1000.0 Hz
 xplotter_layout: None
>


From now we will be working on the resampled data. Data with the 5000 Hz sample rate can he archived as they no longer will be used.

## Converting to MEG BIDS format

Read in the resampled data

In [6]:
del raw, raw_resampled
raw = mne.io.read_raw(raw_resampled_fname)

Opening raw data file /home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_rs.fif...


/tmp/ipykernel_39257/1265112872.py:2: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(raw_resampled_fname)


    Range : 0 ... 1772379 =      0.000 ...  1772.379 secs
Ready.


### Finding triggers and write a FIF file

The BIDS data will include trigger information. As a first step all the events extract from the trigger channel and also included in an additional FIF file.

In [7]:
events = mne.find_events(raw, stim_channel="di32")
mne.write_events(event_fname, events, overwrite=True)

1744 events found on stim channel di32
Event IDs: [   1    2    6    7   50  400  500  800  900 1000 1100 2000 2100]


/tmp/ipykernel_39257/199170786.py:2: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_event.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  mne.write_events(event_fname, events, overwrite=True)


### Naming the events according to trigger information

The trigger values will now be assigned labels. This will be dependent on specific design of the study and care should be
taken to assing informative labels.

In [8]:
event_dict = {
    "cue_right": 1,
    "cue_left": 2,
    "trial_start":50,
    "stim_onset":400,
    "catch_onset":500,
    "dot_right": 6,
    "dot_left": 7,
    "resp": 800,
    "blk_start": 900,
    "blk_end":1000,
    "exp_end":1100,
    "reston":2000,
    "restoff":2100
}

### Writing the BIDS file

For the BIDS conversion several parameters needs to be defined according to the subjection and session. Subsequently the BIDS structure can be written.

In [9]:
raw.info["line_freq"] = 50

subject = '01'
session = '01'
task = 'SpAtt'
run = '01'

bids_path = BIDSPath(
    subject=subject, 
    session=session, 
    task=task, 
    run=run, 
    datatype="meg", 
    root=bids_folder
)
write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=event_fname,
    event_id=event_dict,
    overwrite=True,
)

Opening raw data file /home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_rs.fif...


/tmp/ipykernel_39257/172594259.py:16: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  write_raw_bids(


    Range : 0 ... 1772379 =      0.000 ...  1772.379 secs
Ready.
Writing '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/README'...
Writing '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/participants.tsv'...
Writing '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/participants.json'...
Writing '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json'...
Writing '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json'...
Used Annotations descriptions: ['blk_end', 'blk_start', 'catch_onset', 'cue_left', 'cue_right', 'dot_left', 'dot_right', 'exp_end', 'resp', 'restoff', 'reston', 'stim_onset', 'trial_start']
Writing '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-01_events.tsv'...
Writing '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/su

/tmp/ipykernel_39257/172594259.py:16: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Main_event.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  write_raw_bids(


Closing /home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-01_meg.fif
[done]
Writing '/home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv'...
Wrote /home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv entry with meg/sub-01_ses-01_task-SpAtt_run-01_meg.fif.


BIDSPath(
root: /home/rakshita/rds_folders/Sproject3/OPM_data/Raw_Sub3/Sub3_BIDS
datatype: meg
basename: sub-01_ses-01_task-SpAtt_run-01_meg.fif)

As last check print the structure of the BIDS file

In [ ]:
print_dir_tree(bids_folder)

## References
Niso G, Gorgolewski KJ, Bock E, Brooks TL, Flandin G, Gramfort A, Henson RN, Jas M, Litvak V, T Moreau J, Oostenveld R, Schoffelen JM, Tadel F, Wexler J, Baillet S. MEG-BIDS, the brain imaging data structure extended to magnetoencephalography (2018) Sci Data 5:180110. doi: 10.1038/sdata.2018.110.

Smith SW (1998) The Scientist and Engineer's Guide to Digital Signal Processing. California Technical Pub https://www.dspguide.com/pdfbook.htm
